In [1]:
## initialization 1214

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import constrNMPy
import os 
import warnings

from matplotlib import gridspec
from scipy import sparse, special, stats

%config InlineBackend.figure_format = 'retina'
warnings.simplefilter(action="ignore", category=FutureWarning)

# color pallette
[gr,ye,re,bl,pu,ir,ak] = ['#8ECFC9', '#FFBE7A', '#FA7F6F', '#82B0D2','#BEB8DC', '#E7DAD2','#999999']
[vio, grb, lig, sil, aqua] = ['#8c84cf','#6699CC','#66CC99','#C0C0C0','#6db3bc']
# set the style of the plots
az.style.use("arviz-white")

# Define criterion & d' function

import scipy.stats as stats
def criterion(hit, FA):
    return -0.5*(stats.norm.ppf(hit) + stats.norm.ppf(FA))

def dprime(hit, FA):
    return (stats.norm.ppf(hit) - stats.norm.ppf(FA))/np.sqrt(2)

def hit_fa(df):
    hit = df[(df['StimSeq'] == 45)]['Accuracy']
    fa = 1 - df[(df['StimSeq'] == -45)]['Accuracy']
    return hit, fa

def hit_fa_motor(df):
    hit = df[(df['StimSeq'] == df['ProbeOriRight'])]['Accuracy']
    fa = 1 - df[(df['StimSeq'] == df['ProbeOriLeft'])]['Accuracy']
    return hit, fa

from statistics import mean, stdev
from math import sqrt

# 1 Data Preprocessing
Loading, Screening and Contrast Shifting

In [2]:
import dill

# load data
with open('./all_data.pkl', 'rb') as f:
    all_data = dill.load(f)  # obj = dill.load(file)
    
maxsubj = 32; maxblock = 20; maxtrial = 50

# Screening data by 3 SD from the mean RT
for iS in range(1, maxsubj + 1):
    subject_data = all_data[iS]  # all blocks for this subject
    temp_RT = []
    temp_count = 0
    ## calculate RT mean and SD for each subject
    for iB in range(1, maxblock + 1):
        key = f"{iB}_block"
        block_data = subject_data[iB - 1]  # read current block
        temp_RT.append(block_data['RT'])
    temp_RT = np.concatenate(temp_RT)
    ## Screening data by 3 SD from the mean
    for iB in range(1, maxblock + 1):
        key = f"{iB}_block"
        block_data = subject_data[iB - 1]
        all_data[iS][iB - 1]['Qualified'] = np.logical_and(block_data['RT'] > mean(temp_RT) - 3 * stdev(temp_RT),
                                                           block_data['RT'] < mean(temp_RT) + 3 * stdev(temp_RT))
        temp_count += np.sum(all_data[iS][iB - 1]['Qualified'])
    print(1000 - temp_count, end=' ') 
    
# Shifting the ContrastSeq to one-ahead
for iS in range(1, maxsubj + 1):
    subject_data = all_data[iS]  # all blocks for this subject
    for iB in range(1, maxblock + 1):
        key = f"{iB}_block"
        block_data = subject_data[iB - 1]  # read current block
        block_data['ContrastSeq'] = block_data['ContrastSeq'].shift(1)


16 10 3 16 16 17 11 19 14 12 15 15 18 6 18 13 23 16 8 12 22 16 18 22 14 12 18 16 20 23 9 12 

# 2 Percept and Motor Serial Dependence

In this section, we first reintroduce the perceptual and motor serial dependence as baseline.

Then, we track which kind of trials are more prone to serial dependence.
- Same Stimulus Reactivation?
- Same Response Key Pattern?
- Same Key Response?

## 2.1 SD with Feedbacks

### 2.1.1 Perceptual Serial Dependence

In [4]:
maxsubj = 32 ;maxblock = 20 ;maxtrial = 50 ;num_backs = 10  # 1-back to 10-back

# Dictionary to store data for different back levels
SD_back_Percept_Y = {f"{i}_back": [] for i in range(1, num_backs + 1)} # 1-10 back, with correct feedback (correct response)
SD_back_Percept_N = {f"{i}_back": [] for i in range(1, num_backs + 1)} # 1-10 back, with incorrect feedback (incorrect response)
criterion_Percept_Y_45 = {f"{i}_back": [] for i in range(1, num_backs + 1)} # 1-10 back, with correct feedback (correct response)
criterion_Percept_N_45 = {f"{i}_back": [] for i in range(1, num_backs + 1)} # 1-10 back, with incorrect feedback (incorrect response)
criterion_Percept_Y_n45 = {f"{i}_back": [] for i in range(1, num_backs + 1)} # 1-10 back, with correct feedback (correct response)
criterion_Percept_N_n45 = {f"{i}_back": [] for i in range(1, num_backs + 1)} # 1-10 back, with incorrect feedback (incorrect response)

def process_data_percept_FB(data, back, feedback):
    """Process data and return hits and false alarms"""
    hit_L, fa_L = hit_fa(data[(data['RespOri'].shift(back) == 45) & (data['RespLorR'] == -1) & (data['Accuracy'].shift(back) == feedback) & (data['Qualified'] == True)])
    hit_R, fa_R = hit_fa(data[(data['RespOri'].shift(back) == 45) & (data['RespLorR'] == 1) & (data['Accuracy'].shift(back) == feedback) & (data['Qualified'] == True)])
    hit_nL, fa_nL = hit_fa(data[(data['RespOri'].shift(back) == -45) & (data['RespLorR'] == -1) & (data['Accuracy'].shift(back) == feedback) & (data['Qualified'] == True)])
    hit_nR, fa_nR = hit_fa(data[(data['RespOri'].shift(back) == -45) & (data['RespLorR'] == 1) & (data['Accuracy'].shift(back) == feedback) & (data['Qualified'] == True)])
    return (hit_L, fa_L, hit_R, fa_R, hit_nL, fa_nL, hit_nR, fa_nR)

FB_types = [1, 0]  # 1: correct feedback; 0: incorrect feedback

for iS in range(1, maxsubj + 1):
    subject_data = all_data[iS]  # Get all block data for the current subject
    
    # Create a temporary dictionary to store data for each back level
    temp_data_Y = {f"{i}_back": {"Hit_L": [], "FA_L": [], "Hit_R": [], "FA_R": [], "Hit_nL": [], "FA_nL": [], "Hit_nR": [], "FA_nR": []} for i in range(1, num_backs + 1)}
    temp_data_N = {f"{i}_back": {"Hit_L": [], "FA_L": [], "Hit_R": [], "FA_R": [], "Hit_nL": [], "FA_nL": [], "Hit_nR": [], "FA_nR": []} for i in range(1, num_backs + 1)}
    temp_data = {1: temp_data_Y, 0: temp_data_N}
    
    for iFB in FB_types:
        feedback = iFB
        for iB in range(1, maxblock + 1):
            block_data = subject_data[iB - 1] # Get data for the current block
            for back in range(1, num_backs + 1):
                hits_fa = process_data_percept_FB(block_data, back, feedback)
                temp_data[feedback][f"{back}_back"]["Hit_L"].append(hits_fa[0])
                temp_data[feedback][f"{back}_back"]["FA_L"].append(hits_fa[1])
                temp_data[feedback][f"{back}_back"]["Hit_R"].append(hits_fa[2])
                temp_data[feedback][f"{back}_back"]["FA_R"].append(hits_fa[3])
                temp_data[feedback][f"{back}_back"]["Hit_nL"].append(hits_fa[4])
                temp_data[feedback][f"{back}_back"]["FA_nL"].append(hits_fa[5])
                temp_data[feedback][f"{back}_back"]["Hit_nR"].append(hits_fa[6])
                temp_data[feedback][f"{back}_back"]["FA_nR"].append(hits_fa[7])

    # Calculate the criterion score for each back level and add it to SD_back_Percept_Y and SD_back_Percept_N
    for iFB in FB_types:
        feedback = iFB
        for back in range(1, num_backs + 1):
            key = f"{back}_back"
            hits_L = np.concatenate(temp_data[feedback][key]["Hit_L"])
            fas_L = np.concatenate(temp_data[feedback][key]["FA_L"])
            hits_R = np.concatenate(temp_data[feedback][key]["Hit_R"])
            fas_R = np.concatenate(temp_data[feedback][key]["FA_R"])
            hits_nL = np.concatenate(temp_data[feedback][key]["Hit_nL"])
            fas_nL = np.concatenate(temp_data[feedback][key]["FA_nL"])
            hits_nR = np.concatenate(temp_data[feedback][key]["Hit_nR"])
            fas_nR = np.concatenate(temp_data[feedback][key]["FA_nR"])
            cri_L = criterion(np.mean(hits_L)*0.99+0.005, np.mean(fas_L)*0.99+0.005)
            cri_R = criterion(np.mean(hits_R)*0.99+0.005, np.mean(fas_R)*0.99+0.005)
            cri_nL = criterion(np.mean(hits_nL)*0.99+0.005, np.mean(fas_nL)*0.99+0.005)
            cri_nR = criterion(np.mean(hits_nR)*0.99+0.005, np.mean(fas_nR)*0.99+0.005)
            cri_45 = np.mean([cri_L, cri_R])
            cri_n45 = np.mean([cri_nL, cri_nR])
            SD_back_percept = np.mean([cri_nL - cri_L, cri_nR - cri_R])
            if feedback == 1:
                SD_back_Percept_Y[key].append(SD_back_percept)
                criterion_Percept_Y_45[key].append(cri_45)
                criterion_Percept_Y_n45[key].append(cri_n45)
            else:
                SD_back_Percept_N[key].append(SD_back_percept)
                criterion_Percept_N_45[key].append(cri_45)
                criterion_Percept_N_n45[key].append(cri_n45)
    print('done ', end="@")

SD_nbacks_Percept_Y = pd.DataFrame(SD_back_Percept_Y)
SD_nbacks_Percept_N = pd.DataFrame(SD_back_Percept_N)
criterion_nbacks_Percept_Y_45 = pd.DataFrame(criterion_Percept_Y_45)
criterion_nbacks_Percept_Y_n45 = pd.DataFrame(criterion_Percept_Y_n45)
criterion_nbacks_Percept_N_45 = pd.DataFrame(criterion_Percept_N_45)
criterion_nbacks_Percept_N_n45 = pd.DataFrame(criterion_Percept_N_n45)

done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @

### 2.1.2 Motor Serial Dependence

In [5]:
maxsubj = 32; maxblock = 20 ;maxtrial = 50 ;num_backs = 10  # N-back

def process_data_motor_FB(data, back, feedback):
    """Return Hs and FAs"""
    hit_45L, fa_45L = hit_fa_motor(data[(data['RespOri'] == 45) & (data['RespLorR'].shift(back) == -1) & (data['Accuracy'].shift(back) == feedback) & (data['Qualified'] == True)])
    hit_n45L, fa_n45L = hit_fa_motor(data[(data['RespOri'] == -45) & (data['RespLorR'].shift(back) == -1) & (data['Accuracy'].shift(back) == feedback) & (data['Qualified'] == True)])
    hit_45R, fa_45R = hit_fa_motor(data[(data['RespOri'] == 45) & (data['RespLorR'].shift(back) == 1) & (data['Accuracy'].shift(back) == feedback) & (data['Qualified'] == True)])
    hit_n45R, fa_n45R = hit_fa_motor(data[(data['RespOri'] == -45) & (data['RespLorR'].shift(back) == 1) & (data['Accuracy'].shift(back) == feedback) & (data['Qualified'] == True)])
    return (hit_45L, fa_45L, hit_n45L, fa_n45L, hit_45R, fa_45R, hit_n45R, fa_n45R)

SD_back_motor_Y = {f"{i}_back": [] for i in range(1, num_backs + 1)} # Correct feedback
SD_back_motor_N = {f"{i}_back": [] for i in range(1, num_backs + 1)} # Incorrect feedback
Criterion_after_L_Y = {f"{i}_back": [] for i in range(1, num_backs + 1)} # Correct feedback
Criterion_after_L_N = {f"{i}_back": [] for i in range(1, num_backs + 1)} # Incorrect feedback
Criterion_after_R_Y = {f"{i}_back": [] for i in range(1, num_backs + 1)} # Correct feedback
Criterion_after_R_N = {f"{i}_back": [] for i in range(1, num_backs + 1)} # Incorrect feedback

FB_types = [1, 0]  # 1: correct feedback; 0: incorrect feedback

for iS in range(1, maxsubj + 1):
    subject_data = all_data[iS]  # Get Subject Data
    temp_data_Y = {f"{i}_back": {"Hit_45L": [], "FA_45L": [], "Hit_n45L": [], "FA_n45L": [], "Hit_45R": [], "FA_45R": [], "Hit_n45R": [], "FA_n45R": []} for i in range(1, num_backs + 1)}
    temp_data_N = {f"{i}_back": {"Hit_45L": [], "FA_45L": [], "Hit_n45L": [], "FA_n45L": [], "Hit_45R": [], "FA_45R": [], "Hit_n45R": [], "FA_n45R": []} for i in range(1, num_backs + 1)}
    temp_data = {1: temp_data_Y, 0: temp_data_N}
    
    for iFB in FB_types:
        feedback = iFB
        for iB in range(1, maxblock + 1):
            block_data = subject_data[iB - 1]  # Get Block Data
            for back in range(1, num_backs + 1):
                hits_fa = process_data_motor_FB(block_data, back, feedback)
                temp_data[feedback][f"{back}_back"]["Hit_45L"].append(hits_fa[0])
                temp_data[feedback][f"{back}_back"]["FA_45L"].append(hits_fa[1])
                temp_data[feedback][f"{back}_back"]["Hit_n45L"].append(hits_fa[2])
                temp_data[feedback][f"{back}_back"]["FA_n45L"].append(hits_fa[3])
                temp_data[feedback][f"{back}_back"]["Hit_45R"].append(hits_fa[4])
                temp_data[feedback][f"{back}_back"]["FA_45R"].append(hits_fa[5])
                temp_data[feedback][f"{back}_back"]["Hit_n45R"].append(hits_fa[6])
                temp_data[feedback][f"{back}_back"]["FA_n45R"].append(hits_fa[7])

    for iFB in FB_types:
        feedback = iFB
        for back in range(1, num_backs + 1):
            key = f"{back}_back"
            hits_45L = np.concatenate(temp_data[feedback][key]["Hit_45L"])
            fas_45L = np.concatenate(temp_data[feedback][key]["FA_45L"])
            hits_n45L = np.concatenate(temp_data[feedback][key]["Hit_n45L"])
            fas_n45L = np.concatenate(temp_data[feedback][key]["FA_n45L"])
            hits_45R = np.concatenate(temp_data[feedback][key]["Hit_45R"])
            fas_45R = np.concatenate(temp_data[feedback][key]["FA_45R"])
            hits_n45R = np.concatenate(temp_data[feedback][key]["Hit_n45R"])
            fas_n45R = np.concatenate(temp_data[feedback][key]["FA_n45R"])
            
            cri_45L = criterion(np.mean(hits_45L)*0.99+0.005, np.mean(fas_45L)*0.99+0.005)
            cri_n45L = criterion(np.mean(hits_n45L)*0.99+0.005, np.mean(fas_n45L)*0.99+0.005)
            cri_45R = criterion(np.mean(hits_45R)*0.99+0.005, np.mean(fas_45R)*0.99+0.005)
            cri_n45R = criterion(np.mean(hits_n45R)*0.99+0.005, np.mean(fas_n45R)*0.99+0.005)
            cri_L = np.mean([cri_45L, cri_n45L])
            cri_R = np.mean([cri_45R, cri_n45R])
            
            SD_back_motor_i = np.mean([cri_45L - cri_45R, cri_n45L - cri_n45R])
            if feedback == 1:
                SD_back_motor_Y[key].append(SD_back_motor_i)
                Criterion_after_L_Y[key].append(cri_L)
                Criterion_after_R_Y[key].append(cri_R)
            else:
                SD_back_motor_N[key].append(SD_back_motor_i)
                Criterion_after_L_N[key].append(cri_L)
                Criterion_after_R_N[key].append(cri_R)

    print('done ', end="@")

SD_nbacks_motor_Y = pd.DataFrame(SD_back_motor_Y)
SD_nbacks_motor_N = pd.DataFrame(SD_back_motor_N)
Criterions_after_L_Y = pd.DataFrame(Criterion_after_L_Y)
Criterions_after_L_N = pd.DataFrame(Criterion_after_L_N)
Criterions_after_R_Y = pd.DataFrame(Criterion_after_R_Y)
Criterions_after_R_N = pd.DataFrame(Criterion_after_R_N)

done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @done @

## 2.2 Locating Different Types of Trials

### 2.2.1 Same Stimulus Reactivation

This part, we only look into successfive trials, which is 1-back SD.

之前的分析只是固定看是否存在刺激历史效应，但是没有考虑到底SD是在那种情况下被诱发的。

这一板块，从 Reactivation 的角度去思考是否对于和之前刺激一样的反应会有更高的选中率。

In [6]:
# Finding Key Trials
maxsubj = 32; maxblock = 20; maxtrial = 50; num_backs = 10  # 1-back to 10-back

def same_stim_trials(subject_data, ori):
    """Return the count of same stim trials in each block"""
    trials_count = []
    trials = []
    for iB in range(1, maxblock + 1):
        block_data = subject_data[iB - 1]
        trials.append(block_data[
            (block_data['StimSeq'] == ori) &
            (block_data['StimSeq'].shift(1) == ori) &
            (block_data['Qualified'] == True) ])
        trials_count.append(len(trials[iB - 1]))
    trials = pd.concat(trials)
    count = np.sum(trials_count)
    acc = np.mean(trials['Accuracy'])
    return count, trials, acc

def diff_stim_trials(subject_data, ori):
    """Return the count of diff stim trials in each block"""
    trials_count = []
    trials = []
    for iB in range(1, maxblock + 1):
        block_data = subject_data[iB - 1]
        trials.append(block_data[
            (block_data['StimSeq'] == ori) &
            (block_data['StimSeq'].shift(1) == -ori) &
            (block_data['Qualified'] == True) ])
        trials_count.append(len(trials[iB - 1]))
    trials = pd.concat(trials)
    count = np.sum(trials_count)
    acc = np.mean(trials['Accuracy'])
    return count, trials, acc

def same_resp_trials(subject_data, ori):
    """Return the count of same stim as prev ori trials in each block"""
    trials_count = []
    trials = []
    for iB in range(1, maxblock + 1):
        block_data = subject_data[iB - 1]
        trials.append(block_data[
            (block_data['StimSeq'] == ori) &
            (block_data['RespOri'].shift(1) == ori) &
            (block_data['Qualified'] == True) ])
        trials_count.append(len(trials[iB - 1]))
    trials = pd.concat(trials)
    count = np.sum(trials_count)
    acc = np.mean(trials['Accuracy'])
    return count, trials, acc

def diff_resp_trials(subject_data, ori):
    """Return the count of diff stim as prev ori trials in each block"""
    trials_count = []
    trials = []
    for iB in range(1, maxblock + 1):
        block_data = subject_data[iB - 1]
        trials.append(block_data[
            (block_data['StimSeq'] == ori) &
            (block_data['RespOri'].shift(1) == -ori) &
            (block_data['Qualified'] == True) ])
        trials_count.append(len(trials[iB - 1]))
    trials = pd.concat(trials)
    count = np.sum(trials_count)
    acc = np.mean(trials['Accuracy'])
    return count, trials, acc
    

# print out the number of same stim trials for each subject
# avg_45_same_stim = []; avg_n45_same_stim = []
# for iS in range(1, maxsubj + 1):
#     subject_data = all_data[iS]
#     avg_45_same_stim.append(same_stim_trials(subject_data, 45)[0])
#     avg_n45_same_stim.append(same_stim_trials(subject_data, -45)[0])
# print("avg_45_same_stim: ", np.mean(avg_45_same_stim))
# print("avg_n45_same_stim: ", np.mean(avg_n45_same_stim))

# Now we have to make a table of the mean accuracy for each subject for each trial type
Accuracy_Table  = pd.DataFrame(columns = ['Subject', 'Average', 'Same_Stim_45', 'Same_Stim_n45', 'Diff_Stim_45', 'Diff_Stim_n45', 'Same_Resp_45', 'Same_Resp_n45', 'Diff_Resp_45', 'Diff_Resp_n45'])

def subj_acc_table(subject_data):
    """Return the accuracy table for each subject"""
    temp = pd.DataFrame(columns = ['Subject', 'Average', 'Same_Stim_45', 'Same_Stim_n45', 'Diff_Stim_45', 'Diff_Stim_n45', 'Same_Resp_45', 'Same_Resp_n45', 'Diff_Resp_45', 'Diff_Resp_n45'])
    temp['Subject'] = [iS]
    avg_acc = []
    for iB in range(1, maxblock + 1):
        block_data = subject_data[iB - 1]
        avg_acc.append(np.mean(block_data['Accuracy']))
    temp['Average'] = [np.mean(avg_acc)]
    temp['Same_Stim_45'] = same_stim_trials(subject_data, 45)[2]
    temp['Same_Stim_n45'] = same_stim_trials(subject_data, -45)[2]
    temp['Diff_Stim_45'] = diff_stim_trials(subject_data, 45)[2]
    temp['Diff_Stim_n45'] = diff_stim_trials(subject_data, -45)[2]
    temp['Same_Resp_45'] = same_resp_trials(subject_data, 45)[2]
    temp['Same_Resp_n45'] = same_resp_trials(subject_data, -45)[2]
    temp['Diff_Resp_45'] = diff_resp_trials(subject_data, 45)[2]
    temp['Diff_Resp_n45'] = diff_resp_trials(subject_data, -45)[2]
    return temp

for iS in range(1, maxsubj + 1):
    subject_data = all_data[iS]
    subj_temp = subj_acc_table(subject_data)
    Accuracy_Table = Accuracy_Table.append(subj_temp, ignore_index = True)

In [7]:
# We make another table for combining 45 and n45 trials
Accuracy_Table_Combined = pd.DataFrame(columns = ['Subject', 'Average', 'Same_Stim', 'Diff_Stim', 'Same_Resp', 'Diff_Resp'])

for iS in range(1, maxsubj + 1):
    temp = pd.DataFrame(columns = ['Subject', 'Average', 'Same_Stim', 'Diff_Stim', 'Same_Resp', 'Diff_Resp'])
    temp['Subject'] = [iS]
    temp['Average'] = Accuracy_Table['Average'][iS - 1]
    temp['Same_Stim'] = np.mean([Accuracy_Table['Same_Stim_45'][iS - 1], Accuracy_Table['Same_Stim_n45'][iS - 1]])
    temp['Diff_Stim'] = np.mean([Accuracy_Table['Diff_Stim_45'][iS - 1], Accuracy_Table['Diff_Stim_n45'][iS - 1]])
    temp['Same_Resp'] = np.mean([Accuracy_Table['Same_Resp_45'][iS - 1], Accuracy_Table['Same_Resp_n45'][iS - 1]])
    temp['Diff_Resp'] = np.mean([Accuracy_Table['Diff_Resp_45'][iS - 1], Accuracy_Table['Diff_Resp_n45'][iS - 1]])
    Accuracy_Table_Combined = Accuracy_Table_Combined.append(temp, ignore_index = True)
    
Accuracy_Table_Combined.head()
    

,Subject,Average,Same_Stim,Diff_Stim,Same_Resp,Diff_Resp
0,1,0.755,0.776385,0.747120,0.837500,0.694312
1,2,0.742,0.767250,0.727212,0.775000,0.723525
2,3,0.747,0.753719,0.750337,0.779674,0.725753
3,4,0.743,0.754310,0.745759,0.819660,0.681107
4,5,0.737,0.750602,0.726143,0.798402,0.689038


In [8]:
# Run T-tests on same vs. diff stim trials in Accuracy_Table_Combined
from scipy.stats import ttest_rel
print("Same vs. Diff Stim", "mean: ", np.mean(Accuracy_Table_Combined['Same_Stim']), np.mean(Accuracy_Table_Combined['Diff_Stim']))
print("T-Test: ",ttest_rel(Accuracy_Table_Combined['Same_Stim'], Accuracy_Table_Combined['Diff_Stim']))
print("Same vs. Diff Resp", "mean: ", np.mean(Accuracy_Table_Combined['Same_Resp']), np.mean(Accuracy_Table_Combined['Diff_Resp']))
print("T-Test: ",ttest_rel(Accuracy_Table_Combined['Same_Resp'], Accuracy_Table_Combined['Diff_Resp']))

Same vs. Diff Stim mean:  0.7619319941525586 0.741437534475387
T-Test:  TtestResult(statistic=3.5943165368961747, pvalue=0.0011117602519320766, df=31)
Same vs. Diff Resp mean:  0.7835944995104528 0.7203965703195911
T-Test:  TtestResult(statistic=7.284387343132755, pvalue=3.388894206358187e-08, df=31)


### 2.2.2 Same Response Key Pattern Reactivation

Since we have 2 keys, we have 4 patterns in total. We aim to see if the same pattern would bias to previous key choice.

i.e. if the key pattern is the same/different as previous trial, would it be more likely to stay/switch the key choice?

之前的 Motor Serial Dependence 板块的分析方法往往只看当前的执行动作是否和前一次相关。

但是，这个过程中其实存在着更多的变量，比如给被试呈现的**按键反应的模式**，即是 "/ \" 还是 "\ /"。

如果将SD认为归因为一种记忆的 Reactivation，其实这中呈现给被试的刺激也应该考虑在其中。

In [31]:
all_data[1][1].head()

,ContrastSeq,StimSeq,RespOri,RespLorR,Accuracy,RT,ProbeOriLeft,ProbeOriRight,Qualified
0,NaN,-45,-45,1,1,1.116774,45,-45,True
1,0.148891,-45,-45,1,1,1.000669,45,-45,True
2,0.144204,-45,-45,1,1,1.149377,45,-45,True
3,0.139666,45,45,1,1,1.068230,-45,45,True
4,0.134730,45,45,1,1,1.114755,-45,45,True


In [9]:
# Define a function to find the key trials
def key_choice_stay(subject_data, ProbeOriRight, sameOrdiff):
    """Return the count of key choice stay trials in each block"""
    trials = []
    trials_count = []
    stay_prop = []
    for iB in range(1, maxblock + 1):
        block_data = subject_data[iB - 1]
        key_trials = (block_data[
            (block_data['ProbeOriLeft'] == ProbeOriRight) &
            (block_data['ProbeOriLeft'].shift(1) == sameOrdiff * ProbeOriRight) &
            (block_data['Qualified'] == True) ])
        key_trials_index = key_trials.index
        key_trials_prev = block_data.iloc[key_trials_index - 1]
        stay_prop = np.mean(key_trials['RespLorR'].values == key_trials_prev['RespLorR'].values)
        trials.append(key_trials)
        trials_count.append(len(key_trials))
    trials = pd.concat(trials)
    count = np.sum(trials_count)
    stay = np.mean(stay_prop)
    return count, trials, stay

# Now we make a table of the stay prop for each subject for each trial type

Stay_Table = pd.DataFrame(columns = ['Subject', 'Average', 'Same_Pattern_45_n45', 'Same_Pattern_n45_45','Diff_Pattern_45_n45', 'Diff_Pattern_n45_45'])

def subj_stay_table(subject_data):
    temp = pd.DataFrame(columns = ['Subject', 'Average', 'Same_Pattern_45_n45', 'Same_Pattern_n45_45','Diff_Pattern_45_n45', 'Diff_Pattern_n45_45'])
    temp['Subject'] = [iS]
    avg_stay = []
    for iB in range(1, maxblock + 1):
        block_data = subject_data[iB - 1]
        stay_prop = np.mean(block_data['RespLorR'].values == block_data['RespLorR'].shift(1).values)
        avg_stay.append(stay_prop)
    temp['Average'] = [np.mean(avg_stay)]
    temp['Same_Pattern_45_n45'] = key_choice_stay(subject_data, 45, 1)[2]
    temp['Same_Pattern_n45_45'] = key_choice_stay(subject_data, -45, 1)[2]
    temp['Diff_Pattern_45_n45'] = key_choice_stay(subject_data, 45, -1)[2]
    temp['Diff_Pattern_n45_45'] = key_choice_stay(subject_data, -45, -1)[2]
    return temp

for iS in range(1, maxsubj + 1):
    subject_data = all_data[iS]
    subj_temp = subj_stay_table(subject_data)
    Stay_Table = Stay_Table.append(subj_temp, ignore_index = True)

Stay_Table.head()

,Subject,Average,Same_Pattern_45_n45,Same_Pattern_n45_45,Diff_Pattern_45_n45,Diff_Pattern_n45_45
0,1,0.494,0.538462,0.500000,0.250000,0.583333
1,2,0.440,0.454545,0.636364,0.461538,0.642857
2,3,0.538,0.666667,0.846154,0.333333,0.500000
3,4,0.466,0.666667,0.700000,0.266667,0.333333
4,5,0.481,0.545455,0.909091,0.538462,0.416667


In [10]:
# We make another table for combining 45 and n45 trials
Stay_Table_Combined = pd.DataFrame(columns = ['Subject', 'Average', 'Same_Pattern', 'Diff_Pattern'])

for iS in range(1, maxsubj + 1):
    temp = pd.DataFrame(columns=['Subject', 'Average', 'Same_Pattern', 'Diff_Pattern'])
    temp['Subject'] = [iS]
    temp['Average'] = Stay_Table['Average'][iS - 1]
    temp['Same_Pattern'] = np.mean([Stay_Table['Same_Pattern_45_n45'][iS - 1], Stay_Table['Same_Pattern_n45_45'][iS - 1]])
    temp['Diff_Pattern'] = np.mean([Stay_Table['Diff_Pattern_45_n45'][iS - 1], Stay_Table['Diff_Pattern_n45_45'][iS - 1]])
    Stay_Table_Combined = Stay_Table_Combined.append(temp, ignore_index=True)

Stay_Table_Combined.head()


,Subject,Average,Same_Pattern,Diff_Pattern
0,1,0.494,0.519231,0.416667
1,2,0.440,0.545455,0.552198
2,3,0.538,0.756410,0.416667
3,4,0.466,0.683333,0.300000
4,5,0.481,0.727273,0.477564


In [14]:
# Run T-tests on same vs. diff stim trials in Accuracy_Table_Combined
print("Average", "mean: ", np.mean(Stay_Table_Combined['Average']))
# Single sample t-test on the average stay prop
from scipy.stats import ttest_1samp
print("T-Test: ",ttest_1samp(Stay_Table_Combined['Average'], 0.5))

print("Same vs. Diff Pattern", "mean: ", np.mean(Stay_Table_Combined['Same_Pattern']), np.mean(Stay_Table_Combined['Diff_Pattern']))
print("T-Test: ",ttest_rel(Stay_Table_Combined['Same_Pattern'], Stay_Table_Combined['Diff_Pattern']))

Average mean:  0.47890625000000003
T-Test:  TtestResult(statistic=-3.6990148389499597, pvalue=0.0008367742936440199, df=31)
Same vs. Diff Pattern mean:  0.5671728661963036 0.452924623374991
T-Test:  TtestResult(statistic=4.370973555933828, pvalue=0.00012903251391875576, df=31)


在这里其实我们可以看见，按键反应的部分，其实也是存在吸引性的 Serial Dependence 的，只是在整体上被按键模式的 Switch 倾向所掩盖了。

如果构建一个强化学习的模型，这需要进行 model-based 的构建，因为两种反应模式并不是处在同一个 state 中。